# 2022-12-11

## Performance Summary
Inital p1: 163 ms ± 67.7 ms _* see cell for more info_

Initial p2: 2.01 s ± 6.4 ms

## Initial solution
Woke up att 5:50am (~Midnight UTC-5) and did it on time. 
```
      --------Part 1--------   --------Part 2--------
Day       Time   Rank  Score       Time   Rank  Score
 11   01:02:24   6021      0   03:25:47   8882      0
```

### Puzzle 1

We fetch the monkey data and calculate the new worry levels. We then run the divisibility test on the new worry level and move the item accordingly. We repeat this 20 times as per the assignment.

In [1]:
from collections import defaultdict
monkey_items = defaultdict(list)
monkey_business = defaultdict(int)
data = [x.splitlines() for x in open("data/day-11.txt").read().split("\n\n")]

def p1():
    for monkey in data:
        id = monkey[0][:-1].split(" ")[-1]
        items = eval(monkey[1].split(':')[1])
        if type(items) == int: 
            monkey_items[id] += [items]
        else:
            monkey_items[id] += list(items)
    for round in range(20):
        for monkey in data:
            id = monkey[0][:-1].split(" ")[-1]
            operation = monkey[2].split(" = ")[1][3:].replace("old", "item")
            test = int(monkey[3].split(" ")[-1])
            iftrue = monkey[4].split(" ")[-1]
            iffalse = monkey[5].split(" ")[-1]
            for item in monkey_items[id]:
                monkey_business[id] += 1
                worry_level = eval(f"{item} {operation}")
                monkey_bored_worry_level = worry_level // 3 
                if monkey_bored_worry_level % test == 0:
                    monkey_items[iftrue].append(monkey_bored_worry_level)
                else:
                    monkey_items[iffalse].append(monkey_bored_worry_level)
            monkey_items[id] = []
    top_business = sorted(list(monkey_business.values()))[-2:]
    return top_business[0] * top_business[1]

print("Monkey Business:", p1())

Monkey Business: 112815


### Puzzle 2
So this was an interesting one. Running the above code for 10 000 rounds without the worry level divided by three would result in a program that wouldn't converge for likely many thousand years lmao. 

The difference here is that we have made it more class-like for readability, then the key difference in terms of computational methods is that we have calculated the product of the divisibility terms. We use this product to reset the worry level back to a smaller value that upholds the same divisibility properties.

Example:

$D = {19, 5}, \quad D_{\text{prod}} = 95$

We have our large number:

$x = 2847$

The divisibility properties are:

$x \mod 19 = 16, \quad x \mod 5 = 2$

We can denote these more generally as:

$P = \{x \mod D_i \quad \forall i \in D\}$

We calculate the smaller number that upholds the same divisibility properties, generally: 

$y = x - \lfloor \frac{x}{D_{\text{prod}}}\rfloor \cdot x$

For this example:

$y = 2847 - (\lfloor \frac{2847}{95}\rfloor \cdot 95) = 2847 - (29 \cdot 95) = 92$

$y \mod 19 = 16, \quad y \mod 5 = 2$

Thus we have found a smaller number that upholds the same divisibility properties. We use this to refrain the worry levels to grow too large.

$P_x = P_y$

In [2]:
import numpy as np

data = [x.splitlines() for x in open("data/day-11.txt").read().split("\n\n")]

def p2():

    class Monkey:
        def __init__(self, id):
            self.id = id

    class MonkeyBusiness:
        def __init__(self, data):
            self.data = data
            self.monkies = []
            for monkey_data in self.data:
                m = Monkey(monkey_data[0][:-1].split(" ")[-1])
                items = eval(monkey_data[1].split(':')[1])
                m.items = [items] if type(items) == int else list(items)
                m.operation = monkey_data[2].split(" = ")[1][3:].replace("old", "item")
                m.test = int(monkey_data[3].split(" ")[-1])
                m.iftrue = int(monkey_data[4].split(" ")[-1])
                m.iffalse = int(monkey_data[5].split(" ")[-1])
                m.business = 0
                self.monkies.append(m)
            self.worry_loop = int(np.product([m.test for m in self.monkies]))
        def business(self): return [m.business for m in self.monkies]
        def items(self): return [m.items for m in self.monkies]

    monkey_business = MonkeyBusiness(data)

    for round in range(10000):
        print(round, end='\r')
        for monkey in monkey_business.monkies:
            for item in monkey.items:
                if item >= monkey_business.worry_loop:
                    item -= ((item // monkey_business.worry_loop) * monkey_business.worry_loop)
                monkey.business += 1
                worry_level = eval(f"{item} {monkey.operation}")
                if worry_level % monkey.test == 0:
                    (monkey_business.monkies[monkey.iftrue].items).append(worry_level)
                else:
                    (monkey_business.monkies[monkey.iffalse].items).append(worry_level)
            monkey.items = []
    top_business = sorted(list(monkey_business.business()))[-2:]
    return top_business[0] * top_business[1]

print("Monkey Business:", p2())

Monkey Business: 25738411485


# Learnings

Todays problem was fun, at first, I was afraid of string parsing taking a long time. Although, splitting the data into relevant chunks worked out just fine. Something cool that I learned is the `eval` function in python. It can be used to evaluate strings as python code. 

At first, I thought that maybe the solution to p2 was in some general divisability rule. I gave it serious consideration and tried to understand if there were methods to check for divisability on computational graphs and save a computational graph instead. It wasn't until a long while later that it clicked that I could maintain the divisability properties by calculating the product of the divisability terms and then using that to reset the worry level.

- `eval` function in python

# Runtimes

In [3]:
%%timeit
p1()

The slowest run took 4.33 times longer than the fastest. This could mean that an intermediate result is being cached.
165 ms ± 68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit
p2()

2.03 s ± 27.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
